In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import sys
sys.path.append('../')
sys.path.append('../data_processing/')
sys.path.append('../model/')

In [40]:
import torch
from torch_scatter import scatter_mean, scatter_sum
from torch_geometric.loader import DataLoader, DenseDataLoader
from torch_geometric.utils import to_dense_batch
from data_processing.utils import make_edge_mask

In [6]:
from data_processing.paired_data import CombinedGraphDataset, CombinedSparseGraphDataset
from model.utils.utils_diffusion import append_dims, vp_logs, vp_logsnr, mean_flat, center2zero, center2zero_with_mask, center2zero_combined_graph, sample_zero_center_gaussian, sample_zero_center_gaussian_with_mask

In [7]:
# dataset = CombinedGraphDataset(root='../../data/small_dataset', split='all')
dataset = CombinedSparseGraphDataset(root='../../data/small_dataset', split='all')

In [14]:
data_loader = DataLoader(dataset, batch_size=3, shuffle=False, num_workers=24)

In [15]:
one = next(iter(data_loader))
batch = one
batch

DataBatch(x=[140, 13], pos=[140, 3], target_x=[140, 13], target_pos=[140, 3], Gt_mask=[140], ligand_name=[3], batch=[140], ptr=[4])

In [16]:
# dataset[0].node_mask.shape

In [17]:
one.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [18]:
one.x * ~one.Gt_mask[:, None]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [25]:
# node_mask = batch.node_mask
# edge_mask = batch.edge_mask
# CoM = batch.CoM
# num_nodes = batch.num_nodes

num_nodes = batch.x.size(0)
node_mask = torch.ones([num_nodes], device=batch.x.device, dtype=torch.bool)
batch_info = batch.batch

node_mask = node_mask.unsqueeze(-1)
h0 = batch.x
x0 = batch.pos

hT = batch.target_x
xT = batch.target_pos

Gt_mask = batch.Gt_mask   #.view(node_mask.size(0), node_mask.size(1), -1)

In [26]:
node_mask.size()

torch.Size([140, 1])

In [27]:
x0

tensor([[ -2.0808,  21.8273, -28.2589],
        [ -1.9501,  22.5794, -29.2635],
        [ -2.7814,  23.8029, -29.4013],
        [ -3.6636,  24.2354, -28.3832],
        [ -3.7971,  23.5298, -27.1929],
        [ -4.4254,  25.3948, -28.5608],
        [ -4.3235,  26.1345, -29.7399],
        [ -5.0836,  27.2837, -29.9065],
        [ -3.4573,  25.7181, -30.7504],
        [ -2.6932,  24.5616, -30.5812],
        [ -3.2968,  22.7502, -27.0518],
        [ -5.0150,  27.7801, -30.6982],
        [ -0.9639,  22.2320, -30.3006],
        [ -0.5110,  20.9901, -30.4588],
        [  0.4726,  20.6300, -31.4891],
        [  0.7762,  19.2791, -31.6876],
        [  1.7074,  18.8985, -32.6590],
        [  1.9978,  17.5528, -32.8438],
        [  1.1108,  21.6036, -32.2738],
        [  2.0418,  21.2254, -33.2452],
        [  2.3415,  19.8740, -33.4395],
        [  3.2681,  19.5067, -34.4071],
        [  2.6210,  17.2929, -33.4934],
        [  3.6890,  20.1636, -34.9260],
        [ -2.0866,  21.8168, -28.2756],


In [30]:
batch_info[Gt_mask],batch_info[Gt_mask].size() 

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 torch.Size([70]))

In [29]:
x0[Gt_mask].size()

torch.Size([70, 3])

In [36]:
mean = scatter_mean(x0[Gt_mask], batch_info[Gt_mask], dim=0)
mean[batch_info], mean[batch_info[Gt_mask]]

(tensor([[ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.0007,  22.2853, -30.8755],
         [ -1.00

In [37]:
x = x0 - mean[batch_info]
x

tensor([[-1.0801e+00, -4.5797e-01,  2.6166e+00],
        [-9.4941e-01,  2.9413e-01,  1.6120e+00],
        [-1.7807e+00,  1.5176e+00,  1.4742e+00],
        [-2.6629e+00,  1.9501e+00,  2.4923e+00],
        [-2.7964e+00,  1.2445e+00,  3.6826e+00],
        [-3.4247e+00,  3.1095e+00,  2.3147e+00],
        [-3.3228e+00,  3.8492e+00,  1.1356e+00],
        [-4.0829e+00,  4.9984e+00,  9.6902e-01],
        [-2.4566e+00,  3.4328e+00,  1.2512e-01],
        [-1.6925e+00,  2.2763e+00,  2.9432e-01],
        [-2.2961e+00,  4.6493e-01,  3.8237e+00],
        [-4.0143e+00,  5.4948e+00,  1.7732e-01],
        [ 3.6787e-02, -5.3270e-02,  5.7492e-01],
        [ 4.8969e-01, -1.2952e+00,  4.1672e-01],
        [ 1.4733e+00, -1.6553e+00, -6.1358e-01],
        [ 1.7769e+00, -3.0062e+00, -8.1208e-01],
        [ 2.7081e+00, -3.3868e+00, -1.7835e+00],
        [ 2.9985e+00, -4.7325e+00, -1.9683e+00],
        [ 2.1115e+00, -6.8167e-01, -1.3983e+00],
        [ 3.0425e+00, -1.0599e+00, -2.3697e+00],
        [ 3.3422e+00

In [ ]:
def center2zero_sparse_graph(x, Gt_mask, batch_info, mode='GT'):
    if mode == 'GT':
        mean = scatter_mean(x[Gt_mask], batch_info[Gt_mask], dim=0)
        x = x - mean[batch_info]
        return x
    elif mode == 'individual':
        raise NotImplementedError
    else:
        raise NotImplementedError
        

In [22]:
x0_zerocenter = center2zero_combined_graph(x0, node_mask, Gt_mask)
x0_zerocenter

tensor([[[-0.1216],
         [-0.1233],
         [-0.1197],
         [-0.1116],
         [-0.1066],
         [-0.1084],
         [-0.1133],
         [-0.1101],
         [-0.1213],
         [-0.1245],
         [-0.1085],
         [-0.1133],
         [-0.1290],
         [-0.1426],
         [-0.1484],
         [-0.1662],
         [-0.1722],
         [-0.1899],
         [-0.1366],
         [-0.1425],
         [-0.1603],
         [-0.1662],
         [-0.1940],
         [-0.1582],
         [-0.1221],
         [-0.1359],
         [-0.1167],
         [-0.1164],
         [-0.1365],
         [-0.1163],
         [-0.1165],
         [-0.1363],
         [-0.1157],
         [-0.1164],
         [-0.1363],
         [-0.1361],
         [-0.1291],
         [-0.1427],
         [-0.1539],
         [-0.1543],
         [-0.1540],
         [-0.1362],
         [-0.1544],
         [-0.1540],
         [-0.1542],
         [-0.1661],
         [-0.1360],
         [-0.1360],
         [-0.1220],
         [-0.1222],


AssertionError: 

In [12]:
x0_zerocenter[0][:24], x0_zerocenter[0][24:48]

(tensor([[ -0.1661, -22.7235,  33.5140],
         [ -0.0354, -21.9714,  32.5094],
         [ -0.8667, -20.7479,  32.3717],
         [ -1.7489, -20.3154,  33.3897],
         [ -1.8824, -21.0210,  34.5800],
         [ -2.5107, -19.1560,  33.2122],
         [ -2.4088, -18.4163,  32.0331],
         [ -3.1689, -17.2671,  31.8664],
         [ -1.5426, -18.8327,  31.0225],
         [ -0.7785, -19.9892,  31.1917],
         [ -1.3821, -21.8006,  34.7211],
         [ -3.1003, -16.7707,  31.0747],
         [  0.9508, -22.3188,  31.4723],
         [  1.4037, -23.5607,  31.3141],
         [  2.3873, -23.9208,  30.2838],
         [  2.6909, -25.2717,  30.0854],
         [  3.6221, -25.6523,  29.1139],
         [  3.9125, -26.9980,  28.9291],
         [  3.0255, -22.9472,  29.4991],
         [  3.9565, -23.3254,  28.5277],
         [  4.2562, -24.6768,  28.3335],
         [  5.1828, -25.0441,  27.3658],
         [  4.5357, -27.2579,  28.2795],
         [  5.6037, -24.3872,  26.8470]]),
 tensor([[ -0.

In [26]:
GT_mask = node_mask & (~Gt_mask)
xT_single = x0 * GT_mask
xT_single

tensor([[[ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [ -0.0000,   0.0000,  -0.0000],
         [  0.0000,   0.0000,  -0.0000],
         [  0.0000,   0.0000,  -0.0000],
         [  0.0000,   0.0000,  -0.0000],
         [  0.0000,   0.0000,  -0.0000],
         [  0.0000,   0.0000,  -0.0000],
         [  0.0000,   0.0000,  -0.0000],
         [  0.0000,   0.0000,  -0.0000],
         [  0.0000,   0.0000,  -0.0000],
         [  0.0000,   0.0000,  -0.0000],
         [  0.0000,   0.0000,  -0.0000],
         [ -2.08

In [19]:
print(Gt_mask.squeeze(-1), node_mask.squeeze(-1), GT_mask.squeeze(-1))

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, F

In [20]:
sum(batch.Gt_mask)

tensor(24)

In [25]:
def preprocess(x0, xT, h0, hT, node_mask, Gt_mask=None, num_node=None, batch_info=None, use_mass=False):
    # print(num_node)
    # xT is already included in x0
    x0 = center2zero_combined_graph(x0, node_mask, Gt_mask)
    xT = xT

    # convert the dense graphs into sparse ones

    # x0_ = x0.view(x0.size(0) * x0.size(1), -1)
    # xT_ = xT.view(xT.size(0) * xT.size(1), -1)
    # node_mask_ = node_mask.view(node_mask.size(0) * node_mask.size(1), -1)
    node_mask = node_mask.squeeze(-1)
    # print(node_mask.size(), x0.size(), x0[0].size())
    # x0 = x0_[node_mask_].view(x0.size(0), x0.size(1), -1)
    # xT = xT_[node_mask_].view(xT.size(0), xT.size(1), -1)
    bs = x0.size(0)
    x0_, xT_, h0_, hT_ = [], [], [], []
    sparse_Gt_mask = []
    batch_all = []
    for batch_idx in range(bs):
        x0_.append(x0[batch_idx][node_mask[batch_idx]])
        xT_.append(xT[batch_idx][node_mask[batch_idx]])
        h0_.append(h0[batch_idx][node_mask[batch_idx]])
        hT_.append(hT[batch_idx][node_mask[batch_idx]])
        N = node_mask[batch_idx].sum().item()    # 2 x number of nodes
        # print('number of nodes in the current graph', N)
        Gt_mask_batch = torch.zeros(N)
        # print(Gt_mask_batch.size(), Gt_mask_batch)
        Gt_mask_batch[:(N//2)] = 1
        Gt_mask_batch = Gt_mask_batch.bool()
        sparse_Gt_mask.append(Gt_mask_batch)
        batch_all.append(torch.ones(N, dtype=torch.long) * batch_idx)

    x0 = torch.cat(x0_, dim=0)
    xT = torch.cat(xT_, dim=0)
    h0 = torch.cat(h0_, dim=0)
    hT = torch.cat(hT_, dim=0)
    Gt_mask = torch.cat(sparse_Gt_mask, dim=0)
    batch_info = torch.cat(batch_all, dim=0)
    # x0, xT, h0, hT = x0_, xT_, h0_, hT_
    # print(x0.size(), h0.size(), xT.size(), hT.size(), x0.device, Gt_mask.size())
    return x0, xT, h0, hT, Gt_mask, batch_info

In [26]:
processed_x0, processed_xT, processed_h0, processed_hT, processed_Gt_mask, processed_batch_info = preprocess(x0, xT, h0, hT, node_mask, Gt_mask=Gt_mask, num_node=num_nodes, batch_info=batch_info)

In [27]:
processed_x0

tensor([[ -0.3321, -45.4470,  67.0281],
        [ -0.0707, -43.9428,  65.0189],
        [ -1.7333, -41.4958,  64.7433],
        [ -3.4977, -40.6308,  66.7795],
        [ -3.7647, -42.0420,  69.1601],
        [ -5.0213, -38.3120,  66.4243],
        [ -4.8175, -36.8326,  64.0661],
        [ -6.3377, -34.5342,  63.7329],
        [ -3.0851, -37.6654,  62.0451],
        [ -1.5569, -39.9784,  62.3835],
        [ -2.7641, -43.6012,  69.4423],
        [ -6.2005, -33.5414,  62.1495],
        [  1.9017, -44.6376,  62.9447],
        [  2.8075, -47.1214,  62.6283],
        [  4.7747, -47.8416,  60.5677],
        [  5.3819, -50.5434,  60.1707],
        [  7.2443, -51.3046,  58.2279],
        [  7.8251, -53.9960,  57.8583],
        [  6.0511, -45.8944,  58.9983],
        [  7.9131, -46.6508,  57.0555],
        [  8.5125, -49.3536,  56.6669],
        [ 10.3657, -50.0882,  54.7317],
        [  9.0715, -54.5158,  56.5591],
        [ 11.2075, -48.7744,  53.6939],
        [ -0.0000,   0.0000,  -0.0000],


In [28]:
processed_batch_info

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [29]:
processed_Gt_mask

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, False, False,
        False, False, False, False, False, False, False, False, 

# Bugs
1. VP bridge model keeps generating Nan: check if ***sigma max*** is < 1. This happens because of using the sigma values for VE bridge